In [3]:
import pandas as pd
import os
import numpy as np
import math
import random
import torch
import json
from tqdm import tqdm
from collections import defaultdict
from datetime import datetime

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import BERT4Rec, SASRec
from recbole.trainer import Trainer
from recbole.utils import init_seed
from recbole.utils.case_study import full_sort_topk
from recbole.quick_start.quick_start import load_data_and_model

In [2]:

config_dict = {
    'data_path': './data',  # 데이터셋 폴더가 들어있는 상위 경로입니다.
    'USER_ID_FIELD': 'user_idx',  # 사용자 ID가 저장된 컬럼명입니다.
    'ITEM_ID_FIELD': 'item_idx',  # 아이템(상품, 책 등) ID가 저장된 컬럼명입니다.
    'TIME_FIELD': 'event_time',  # 상호작용(클릭, 구매 등) 시각이 저장된 컬럼명입니다.
    'user_inter_num_interval': "[10,Inf)",  # 최소 5번 이상 상호작용한 사용자만 남깁니다.
    'item_inter_num_interval': "[10,Inf)",  # 최소 5번 이상 등장한 아이템만 남깁니다.
    'load_col': {'inter': ['user_idx', 'item_idx', 'user_session', 'event_time', 'event_type'],
                'item': ['item_idx', 'category_code', 'brand', 'price']
                },  # 불러올 컬럼을 명시합니다.

    'train_batch_size': 4096,  # 학습 시 한 번에 처리할 데이터 샘플 수입니다.
    'embedding_size': 128, 
    'hidden_size': 128,  # 임베딩 및 내부 레이어의 차원 수입니다.
    'n_layers': 2,  # 모델의 레이어(층) 개수입니다.
    'n_heads': 4,  # Self-Attention에서 사용하는 헤드 개수입니다.
    'inner_size': 64,  # Feedforward 네트워크의 내부 차원입니다.
    'hidden_dropout_prob': 0.2,  # 은닉층에 적용할 드롭아웃 비율입니다.
    'attn_dropout_prob': 0.2,  # 어텐션 레이어에 적용할 드롭아웃 비율입니다.
    'hidden_act': 'gelu',  # 은닉층에서 사용할 활성화 함수입니다.
    'layer_norm_eps': 1e-12,  # Layer Normalization에서 사용하는 작은 상수입니다.
    'initializer_range': 0.02,  # 가중치 초기화 시 표준편차입니다.
    'pooling_mode': 'sum',  # 시퀀스 임베딩을 합산(sum) 방식으로 합칩니다.
    'loss_type': 'BPR',  # 학습에 사용할 손실 함수(BPR: 순위 기반 추천)입니다.
    'fusion_type': 'gate',  # 여러 정보를 결합할 때 게이트 방식을 사용합니다.
    'attribute_predictor': 'linear',  # 속성 예측에 사용할 방식을 지정합니다.
    #'epoch': 2,  # 전체 데이터셋을 몇 번 반복해서 학습할지 지정합니다.
    'epochs' : 10, 
    'stopping_step': 5,  # 검증 성능이 5번 연속 개선되지 않으면 학습을 멈춥니다.

    'MAX_ITEM_LIST_LENGTH': 50,  # 사용자별로 최대 50개까지의 시퀀스만 사용합니다.
    'eval_args': {
        'split': {'LS': 'valid_and_test'},  # Leave-Sequence 방식으로 검증/테스트 분할
        'group_by': 'user',  # 사용자별로 데이터를 그룹화해서 평가합니다.
        'order': 'TO',  # 시간순(Time Order)으로 정렬해서 분할합니다.
        'mode': 'uni100'  # 테스트 시 각 정답마다 100개의 negative 아이템을 샘플링합니다.
    },
    'metrics': ['Recall', 'NDCG'],  # 평가 지표로 Recall과 NDCG를 사용합니다.
    'topk': 10,  # 상위 10개 추천 결과만 평가에 사용합니다.
    'valid_metric': 'NDCG@10',  # 검증 기준으로 NDCG@10을 사용합니다.
    # 'checkpoint_dir': '/content'  # (주석 처리됨) 모델 체크포인트 저장 디렉토리입니다.
}

config = Config(model='SASRec',
                config_dict=config_dict,
                dataset='sasrec_data')

In [3]:

dataset = create_dataset(config)

# 데이터셋 전체 정보 출력
print("Dataset Info:")
print(dataset)
print(f"\n데이터셋 이름: {dataset.dataset_name}")
print(f"사용자 수: {dataset.user_num}")
print(f"아이템 수: {dataset.item_num}")
print(f"상호작용 수: {dataset.inter_num}")

train_data, valid_data, test_data = data_preparation(config, dataset)


/data/ephemeral/home/work/python/upstageailab-ocr-recsys-competition-recsys-5/.venv/lib/python3.11/site-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/data/ephemeral/home/work/python/upstageailab-ocr-recsys-competition-recsys-5/.venv/lib/python3.11/site-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior w

Dataset Info:
sasrec_data
The number of users: 238147
Average actions of users: 28.943748792757383
The number of items: 28219
Average actions of items: 244.27096179743427
The number of inters: 6892838
The sparsity of the dataset: 99.89743212992866%
Remain Fields: ['user_idx', 'item_idx', 'user_session', 'event_time', 'event_type', 'category_code', 'brand', 'price']

데이터셋 이름: sasrec_data
사용자 수: 238147
아이템 수: 28219
상호작용 수: 6892838


In [ ]:
print(f"Field 타입들: {dataset.field2type}")
print(f"Sparsity: {dataset.sparsity}")

In [4]:
# 3. 데이터를 로딩.
train_df = pd.read_parquet('./data/train.parquet')

train_df.head()

,user_id,item_id,user_session,event_time,category_code,brand,price,event_type
0,0b517454-e7c3-44ec-8c39-a68ef9c0ec60,18c11cbb-a18d-4a9e-bdea-6abd3f7d3c04,ad97f19a-f5fb-41ea-a7b2-52c21fb37ab2,2019-11-16 16:31:26 UTC,apparel.shoes,kapika,72.05,view
1,215eeee5-f9c5-4213-8641-7561dbdad1b9,47c5a6da-32d0-4a29-8b51-57304f476ded,6058b45b-bdb9-4d6c-b300-42dcb1cb8280,2019-11-04 18:59:50 UTC,apparel.shoes,respect,82.63,view
2,a25bf14a-49ac-49bb-87de-ee6b300f0cc4,a6d915c6-2bb7-4393-a556-c327723d3666,28a8b8e3-b374-435d-9d5d-b96058ecb75b,2019-11-26 09:01:47 UTC,apparel.tshirt,goodloot,24.43,view
3,09ee8591-25e0-4bb4-ae24-c48ed4212e3c,0fd4da5d-989c-4a75-9ace-2b108f834c8c,f2972db7-9916-4a58-b6f9-c76afde6245e,2019-11-15 16:05:34 UTC,apparel.shoes,baden,70.79,view
4,7acf7c81-69f6-4aa8-b19f-8e85aeaffc28,d52d1c91-5534-4de4-aaf1-318e932e10e7,7d46d970-b40e-4a2f-81a7-65bf23aa0aae,2019-11-16 13:14:09 UTC,apparel.shoes,rooman,53.80,view


In [7]:
# 4. 데이터 체크

train_df['event_type'].value_counts()  
#train_df['user_session'].nunique() / len(train_df)

event_type
view        8331873
cart          16362
purchase       2076
Name: count, dtype: int64

In [8]:
# 매핑 사전 정의
event_map = {'view': 1, 'cart': 2, 'purchase': 3}

# type_ratio 컬럼 생성
train_df['type_ratio'] = train_df['event_type'].map(event_map).astype('int')

In [15]:
train_df[train_df['event_type'] == 'purchase'].head()

,user_id,item_id,user_session,event_time,category_code,brand,price,event_type,type_ratio
2660,3308e1fb-9ba3-4d5d-b6d7-b69327d1eaf9,f9c5fb98-9577-44a9-b946-d18bc45b8368,8f310373-d270-41b1-8657-17ad43fd44b8,2019-11-13 13:40:01 UTC,apparel.shoes,nexpero,75.42,purchase,3
16872,5a9f26f1-972d-4918-b1ee-73df61d495a0,13eebd7f-62c2-4859-8396-5f4c5c92410c,deb74236-4182-48b7-9d01-0fa77f69b4fc,2019-11-16 14:29:53 UTC,apparel.shoes,respect,84.43,purchase,3
35132,174c647c-663d-418a-9f32-223cfe80e823,0dd57aac-9857-4749-b636-61e40132d609,670edfa2-98b0-463e-a553-1e72e6a25537,2019-11-17 10:16:37 UTC,apparel.shoes,respect,82.63,purchase,3
62514,0cd7faa5-eda0-4544-8d7c-4fb43f41eb65,b6a07470-4084-40c3-9349-7e1275cc1150,89df019f-d1b4-43fd-b04f-c3a51ce9bcc4,2019-11-17 06:54:27 UTC,apparel.shoes,baden,71.82,purchase,3
84596,d7fb91c9-b1ad-4224-b109-7c5f3a936715,fb5a01f5-7261-4de0-94d1-9d75d4792185,6167e9f8-5009-4a1a-8789-efe5e29f4431,2019-11-17 05:48:23 UTC,apparel.shoes,baden,86.23,purchase,3


In [5]:
# 필요한 컬럼만 추출
data = train_df[['user_id', 'item_id', 'event_type', 'event_time']]
# event_time 오름차순 정렬
data = data.sort_values(['user_id', 'item_id', 'event_time'])

In [8]:
data = data.sort_values(['event_time'])

data.tail()

,user_id,item_id,event_type,event_time
7131441,0d4a7ff3-0647-41a8-9176-3775747c10a0,b092fe8f-6252-4a33-ba99-7e4ad94c5a49,view,2020-02-29 23:57:34 UTC
7388009,1ceab7d4-aa38-449e-bdd3-3f96230c2d8d,b91deebf-3570-4ad8-91f0-c6186d94952a,view,2020-02-29 23:58:52 UTC
6846768,497f6916-d368-4d19-b58c-30a0ef456523,d347a160-b39d-4819-bd9f-858df21b3039,view,2020-02-29 23:59:02 UTC
7718564,1b489326-9e05-492e-bdcd-2c52c586c4db,5b071914-719c-4a73-bbd1-ab8100ba1eb2,view,2020-02-29 23:59:28 UTC
8175077,d90757c4-2138-472d-8738-f37a5f98b267,fa8de887-2e69-4ed0-8aaf-9c984c4ab9e1,view,2020-02-29 23:59:33 UTC


In [28]:
# 'view' 이벤트만 1, 나머지는 0으로 표시
data['is_view'] = (data['event_type'] == 'view').astype(int)

# 사용자-아이템별로 시간순 누적합
data['view_cnt'] = data.groupby(['user_id', 'item_id'])['is_view'].cumsum()

In [30]:
train_df.head(50)

,user_id,item_id,user_session,event_time,category_code,brand,price,event_type,type_ratio,view_cnt
0,0b517454-e7c3-44ec-8c39-a68ef9c0ec60,18c11cbb-a18d-4a9e-bdea-6abd3f7d3c04,ad97f19a-f5fb-41ea-a7b2-52c21fb37ab2,2019-11-16 16:31:26 UTC,apparel.shoes,kapika,72.05,view,1,1
1,215eeee5-f9c5-4213-8641-7561dbdad1b9,47c5a6da-32d0-4a29-8b51-57304f476ded,6058b45b-bdb9-4d6c-b300-42dcb1cb8280,2019-11-04 18:59:50 UTC,apparel.shoes,respect,82.63,view,1,1
2,a25bf14a-49ac-49bb-87de-ee6b300f0cc4,a6d915c6-2bb7-4393-a556-c327723d3666,28a8b8e3-b374-435d-9d5d-b96058ecb75b,2019-11-26 09:01:47 UTC,apparel.tshirt,goodloot,24.43,view,1,1
3,09ee8591-25e0-4bb4-ae24-c48ed4212e3c,0fd4da5d-989c-4a75-9ace-2b108f834c8c,f2972db7-9916-4a58-b6f9-c76afde6245e,2019-11-15 16:05:34 UTC,apparel.shoes,baden,70.79,view,1,1
4,7acf7c81-69f6-4aa8-b19f-8e85aeaffc28,d52d1c91-5534-4de4-aaf1-318e932e10e7,7d46d970-b40e-4a2f-81a7-65bf23aa0aae,2019-11-16 13:14:09 UTC,apparel.shoes,rooman,53.80,view,1,1
5,7acf7c81-69f6-4aa8-b19f-8e85aeaffc28,2933991a-32a4-4a87-855e-1879a64cd590,7d46d970-b40e-4a2f-81a7-65bf23aa0aae,2019-11-16 13:15:18 UTC,apparel.shoes,respect,64.61,view,1,1
6,ae46c8ec-1747-4d98-b15e-8dfe89f6986c,476e0658-d99b-4ca6-b337-3a0d46018128,be04d3f9-ad4c-47a3-8989-3ab980d5b689,2019-11-21 15:41:18 UTC,apparel.shoes,respect,89.84,view,1,1
7,ae46c8ec-1747-4d98-b15e-8dfe89f6986c,d1f7b907-e3d8-4969-87f8-feebf46b0646,be04d3f9-ad4c-47a3-8989-3ab980d5b689,2019-11-21 15:42:10 UTC,apparel.shoes,respect,69.24,view,1,3
8,ae46c8ec-1747-4d98-b15e-8dfe89f6986c,e77c501d-6972-4aeb-b1d5-6422e2e31705,be04d3f9-ad4c-47a3-8989-3ab980d5b689,2019-11-21 15:42:41 UTC,apparel.shoes,respect,98.84,view,1,5
9,cd397d08-d078-40cf-9c23-a79fee33efd6,266d2131-1420-4921-826c-c95bdf665caa,8d9ad21e-20a5-43d6-92db-e721c3918569,2019-11-30 10:37:18 UTC,apparel.skirt,weekend,432.44,view,1,1


In [21]:
train_df = train_df.merge(
    data[['user_id', 'item_id', 'event_time', 'view_cnt']],
    on=['user_id', 'item_id', 'event_time'],
    how='left'
)

train_df.head()


,user_id,item_id,user_session,event_time,category_code,brand,price,event_type,type_ratio,view_cnt
0,0b517454-e7c3-44ec-8c39-a68ef9c0ec60,18c11cbb-a18d-4a9e-bdea-6abd3f7d3c04,ad97f19a-f5fb-41ea-a7b2-52c21fb37ab2,2019-11-16 16:31:26 UTC,apparel.shoes,kapika,72.05,view,1,1
1,215eeee5-f9c5-4213-8641-7561dbdad1b9,47c5a6da-32d0-4a29-8b51-57304f476ded,6058b45b-bdb9-4d6c-b300-42dcb1cb8280,2019-11-04 18:59:50 UTC,apparel.shoes,respect,82.63,view,1,1
2,a25bf14a-49ac-49bb-87de-ee6b300f0cc4,a6d915c6-2bb7-4393-a556-c327723d3666,28a8b8e3-b374-435d-9d5d-b96058ecb75b,2019-11-26 09:01:47 UTC,apparel.tshirt,goodloot,24.43,view,1,1
3,09ee8591-25e0-4bb4-ae24-c48ed4212e3c,0fd4da5d-989c-4a75-9ace-2b108f834c8c,f2972db7-9916-4a58-b6f9-c76afde6245e,2019-11-15 16:05:34 UTC,apparel.shoes,baden,70.79,view,1,1
4,7acf7c81-69f6-4aa8-b19f-8e85aeaffc28,d52d1c91-5534-4de4-aaf1-318e932e10e7,7d46d970-b40e-4a2f-81a7-65bf23aa0aae,2019-11-16 13:14:09 UTC,apparel.shoes,rooman,53.80,view,1,1


In [26]:
len(train_df[train_df['view_cnt'].isna()])

0